Copyright **`(c)`** 2023 Giovanni Squillero & Edoardo Franco `<giovanni.squillero@polito.it>` `<s310228@studenti.polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [187]:
from itertools import product
from random import random, randint, shuffle, seed, choice
from scipy import sparse
from functools import reduce
from copy import copy

import numpy as np

In [188]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [189]:
num_points = 5000
num_sets = num_points
density = 0.3
max_iter = 10_000
random_restart_prob = 0.2

fitness_call = 0
found_in_tabu_list = 0
restarts = 0

x = make_set_covering_problem(num_points, num_sets, density).toarray()
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [190]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [x[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)]),
        )
    )
    return valid, -cost

def fitness2(state):
    # Number of taken sets
    cost = sum(state)
    
    # Nuber of covered points
    valid = np.sum(
        reduce(
            np.logical_or,
            [x[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)]),
        )
    )
    return valid, -cost

fitness = fitness2

In [191]:
def tweak(state, restarts):
    new_state = copy(state)
    index = randint(0, num_points - 1)
    new_state[index] = not new_state[index]

    return new_state, restarts

In [192]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
print(fitness(current_state))

# ILS + tabu search
tabu_list = []

for step in range(max_iter):
    new_state, restarts = tweak(current_state, restarts)

    while new_state in tabu_list:
        print("Already in tabu list")
        found_in_tabu_list += 1
        new_state, restarts = tweak(current_state, restarts)
        
    if random() < random_restart_prob and fitness_call > 0:
        new_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
        restarts += 1

    fitness_call += 1
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        tabu_list.append(new_state)
        print(f"{fitness_call} -> {fitness(current_state)}")
        
        if fitness(current_state)[0] == num_points:
            break

print(f"fitness calls: {fitness_call}")
print(f"Tabu list hits: {found_in_tabu_list}, tabu list dimension: {len(tabu_list)}")
print(f"Random restarts: {restarts}")

(0, 0)
1 -> (1446, -1)
2 -> (2477, -2)
3 -> (3239, -3)
4 -> (3766, -4)
5 -> (4129, -5)
6 -> (4399, -6)
7 -> (4552, -7)
8 -> (4669, -8)
9 -> (4772, -9)
10 -> (4841, -10)
11 -> (4886, -11)
12 -> (4920, -12)
13 -> (4942, -13)
14 -> (4956, -14)
15 -> (4973, -15)
17 -> (4984, -16)
18 -> (4990, -17)
19 -> (4994, -18)
20 -> (4996, -19)
22 -> (4997, -20)
24 -> (4998, -21)
28 -> (4999, -22)
31 -> (5000, -23)
fitness calls: 31
Tabu list hits: 0, tabu list dimension: 23
Random restarts: 4
